In [ ]:
#https://www.youtube.com/watch?v=kJbxoyLZGwg
#Datos: Abundance (Scaled) de cada canal y para cada par de condiciones (ej. Cre ctrl y Cre AKI tendría 8 columnas de abundancias)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests

In [ ]:
#Abro el .xlsx y selecciono las hojas, filas y columnas que me interesen. En este caso la de p-péptidos
file=pd.ExcelFile('Quantification_TMT-phospho_4conditions-mouse_FC-arranged.xlsx')
#cargo la hoja de p-péptidos
df_sheet1=file.parse('Pep_dif-phos-5%_CreAKI_CreCtrl')
df_sheet2=file.parse('Pep_dif-phos-5%_NapiAKI_NapCtrl')
df_sheet3=file.parse('Pep_dif-phos-5%_CreAKI_NapiCtrl')
df_sheet4=file.parse('Pep_dif-phos-5%_NapiAKI_CreAKI')
df_sheet5=file.parse('Pep_dif-phos-5%_NapiAKI_CreCtrl')
df_sheet6=file.parse('Pep_dif-phos-5%_NapiCtr_CreCtrl')
df_sheet_list=[df_sheet1,df_sheet2,df_sheet3,df_sheet4,df_sheet5,df_sheet6]

In [ ]:
#Función para obtener la lista de modificaciones de las 20 prot. *identificadas* y con *mayor* ratio (log2)
def get_pos_mod_sites(df_sheet):
    cols=['Modifications','Master Protein Accessions','Master Protein Descriptions','Modifications in Proteins']
    df=df_sheet[cols]
    #print(df)
    pattern1=r'(?<=\[| )[STY]\d+' #que aparezca S, T o Y con "[" o un " " antes *y* seguida de algún número (eso significa que se ha podido reconocer el sitio en la proteína y va a aparecer en la columna de Modifications in Proteins)
    pattern2=r'[STY/](?![a-zA-Z0-9])' #que aparezca "/" o S, T, Y *sin* estar seguida de números (esto significa que no se ha podido reconocer el sitio concreto de fosforilación)
    accession=''
    sites_list=[]
    #match_count=1
    row_count=0 #quiero saber en qué fila estoy
    n_i_rows=[] #una lista para recopilar qué filas no tienen una posición concreta asignada
    n_i_accs=[] #lista para recopilar las accesiones/genes sin posición concreta
    skipped_count=0
    for index,row in df.iterrows():
        match1=re.search(pattern1, row['Modifications'])
        match2=re.search(pattern2, row['Modifications'])
        row_30_check=row_count-skipped_count #quiero tener 30 filas con modificaciones exactas. Si reviso 40 filas y 30 no se añaden porque no tienen modificación, solo tendré 10 filas, por lo que necesito que el bucle siga hasta que no haya 30 mods
        if row_30_check<20:
            if "GN=" not in row['Master Protein Descriptions']: #si una de las filas no tiene el nombre del gen
                n_i_rows.append(row_count) #añado la fila en la que no hay GN
                row_count=row_count+1 
                n_i_name=row['Master Protein Descriptions'].split(" OS")[0] #ej.: Splicing factor C9orf78 homolog OS=Mus musculus OX=10090 PE=1 SV=2
                n_i_accs.append(n_i_name)
                skipped_count=skipped_count+1
            elif not match1 and match2:
                n_i_rows.append(row_count) #añado la fila en la que no hay posición exacta de fosforilación
                row_count=row_count+1
                accession=(row['Master Protein Accessions'].split(';')[0])
                #print(accession)
                n_i_accs.append(accession) #añado la accesión que no tiene posición exacta de fosforilación
                #epnd='(epnd)'#"exact position not determined"
                #sites_list.append(epnd)
                #match_count=match_count+1
                skipped_count=skipped_count+1
            elif match1:
                row_count=row_count+1
                accession=(row['Master Protein Accessions'].split(';')[0])
                #print(accession)
                prots_sites_list=row['Modifications in Proteins'].split('];')
                #match_count=match_count+1
                #print(prots_sites_list)
                for element in prots_sites_list:
                    if accession in element:
                        matches=re.findall(pattern1, element)
                        #print(matches)
                        if len(matches)==1:
                            sites='('+matches[0]+')'
                        elif len(matches)>1: 
                            site_count=0
                            #print(len(matches))
                            for site in matches:
                                #print(site_count)
                                if site_count==0:
                                    sites='('+matches[site_count]+','
                                elif site_count!=0 and site_count!=len(matches)-1:
                                    sites=sites+matches[site_count]+','
                                elif site_count==len(matches)-1:
                                    sites=sites+matches[site_count]+')'
                                site_count=site_count+1
                        sites_list.append(sites)
        #print(row_count)
    perc_n_i=(round(100*len(n_i_accs)/row_count,2))
    return sites_list, n_i_rows, n_i_accs, perc_n_i, row_count #match_count debe coincidir con la cantidad de filas en Excel, si no, algo no ha salido bien (revisar)
sites_list, n_i_rows, n_i_accs, perc_n_i, row_count=get_pos_mod_sites(df_sheet2)
print(sites_list)
print(len(sites_list))
print('\n'+'Accesiones/genes con una posición exacta de fosforilación no identificada:'+'\n'+str(n_i_accs)+' ('+str(len(n_i_accs))+')'+'\n')#+'Son '+str(len(n_i_accs))+' y representan un '+str(perc_n_i)+'% de las proteínas.'+'\n')
#print(n_i_accs) #simplemente necesito el número de la fila (la de Excel - 2) de los que no tienen GN o modificación exacta para evitar meter esa fila en un nuevo df
#print(perc_n_i)
#print('Sites count: '+str(match_count))
print(n_i_rows)
print(row_count)

In [ ]:
#Función para crear un df que contenga solo las filas con sitios concretos
def get_20_df(df_sheet):
    col_names=[] #quiero un nuevo df con las mismas columnas que el original; inicio una lista para añadir los nombres de las columnas del df original
    for name in df_sheet.columns: #añado el nombre de cada columna del df a la lista de nombres
        col_names.append(name)
    df_20=pd.DataFrame(columns=col_names)
    #print(df_20)
    new_row_count=0 #inicio un contador de filas para saber en qué fila del df estamos
    for index,row in df_sheet.iterrows():
        if new_row_count<row_count:
            if new_row_count in n_i_rows:
                #print(row)
                new_row_count=new_row_count+1
            elif new_row_count not in n_i_rows:
                #print(row)
                df_20.loc[new_row_count]=row
                new_row_count=new_row_count+1
        else:
            break
        #print(new_row_count)
    return df_20, col_names
df_20,col_names=get_20_df(df_sheet2)
df_20

In [ ]:
#Función para obtener la lista de modificaciones de las 10 prot. identificadas y con MENOR ratio (log2) [negativo]
def get_neg_mod_sites(df_sheet):
    cols=['Modifications','Master Protein Accessions','Master Protein Descriptions','Modifications in Proteins']
    df=df_sheet[cols]
    #print(df)
    pattern1=r'(?<=\[| )[STY]\d+' #que aparezca S, T o Y con "[" o un " " antes *y* seguida de algún número (eso significa que se ha podido reconocer el sitio en la proteína y va a aparecer en la columna de Modifications in Proteins)
    pattern2=r'[STY/](?![a-zA-Z0-9])' #que aparezca "/" o S, T, Y *sin* estar seguida de números (esto significa que no se ha podido reconocer el sitio concreto de fosforilación)
    accession=''
    sites_list2=[]
    #match_count=1
    row_count=0 #quiero saber cuántas filas llevo parseadas
    reverse_count=len(df)-1 #empiezo una cuenta desde el final de la tabla
    n_i_rows=[] #una lista para recopilar qué filas no tienen una posición concreta asignada
    n_i_accs2=[] #lista para recopilar las accesiones/genes sin posición concreta
    skipped_count=0
    for index,row in df.iloc[::-1].iterrows():
        match1=re.search(pattern1, row['Modifications'])
        match2=re.search(pattern2, row['Modifications'])
        row_10_check=row_count-skipped_count #quiero tener 10 filas con modificaciones exactas. Si reviso 40 filas y 35 no se añaden porque no tienen modificación, solo tendré 5 filas, por lo que necesito que el bucle siga hasta que no haya 30 mods
        if row_10_check<10:
            if "GN=" not in row['Master Protein Descriptions']: #si una de las filas no tiene el nombre del gen
                n_i_rows.append(reverse_count) #añado la fila en la que no hay GN
                row_count+=1
                reverse_count=reverse_count-1
                n_i_name=row['Master Protein Descriptions'].split(" OS")[0] #ej.: Splicing factor C9orf78 homolog OS=Mus musculus OX=10090 PE=1 SV=2
                n_i_accs2.append(n_i_name)
                skipped_count+=1
            elif not match1 and match2:
                n_i_rows.append(reverse_count) #añado la fila en la que no hay GN
                row_count+=1
                reverse_count=reverse_count-1 
                accession=(row['Master Protein Accessions'].split(';')[0])
                #print(accession)
                n_i_accs2.append(accession) #añado la accesión que no tiene posición exacta de fosforilación
                #epnd='(epnd)'#"exact position not determined"
                #sites_list.append(epnd)
                #match_count=match_count+1
                skipped_count=skipped_count+1
            elif match1:
                row_count+=1
                reverse_count=reverse_count-1
                accession=(row['Master Protein Accessions'].split(';')[0])
                #print(accession)
                prots_sites_list=row['Modifications in Proteins'].split('];')
                #match_count=match_count+1
                #print(prots_sites_list)
                for element in prots_sites_list:
                    if accession in element:
                        matches=re.findall(pattern1, element)
                        #print(matches)
                        if len(matches)==1:
                            sites='('+matches[0]+')'
                        elif len(matches)>1: 
                            site_count=0
                            #print(len(matches))
                            for site in matches:
                                #print(site_count)
                                if site_count==0:
                                    sites='('+matches[site_count]+','
                                elif site_count!=0 and site_count!=len(matches)-1:
                                    sites=sites+matches[site_count]+','
                                elif site_count==len(matches)-1:
                                    sites=sites+matches[site_count]+')'
                                site_count=site_count+1
                        sites_list2.append(sites)
        #print(row_count)
    sites_list2=sites_list2[::-1] #le doy la vuelta a la lista porque después la voy a usar en este orden
    perc_n_i=(round(100*len(n_i_accs2)/row_count,2))
    #print(df.iloc[n_i_rows[0]]) #print para comprobar que la primera fila con una fosforilación no identificada de verdad lo es
    return sites_list2, n_i_rows, n_i_accs2, perc_n_i, row_count #match_count debe coincidir con la cantidad de filas en Excel, si no, algo no ha salido bien (revisar)
sites_list2, n_i_rows, n_i_accs2, perc_n_i, row_count=get_neg_mod_sites(df_sheet2)
print(sites_list2)
print(len(sites_list2))
print('\n'+'Accesiones/genes con una posición exacta de fosforilación no identificada:'+'\n'+str(n_i_accs2)+' ('+str(len(n_i_accs2))+')'+'\n')#+'Son '+str(len(n_i_accs2))+' y representan un '+str(perc_n_i)+'% de las proteínas.'+'\n')
#print(n_i_accs2) #simplemente necesito el número de la fila (la de Excel - 2) de los que no tienen GN o modificación exacta para evitar meter esa fila en un nuevo df
#print(perc_n_i)
#print('Sites count: '+str(match_count))
print(n_i_rows)
print(row_count)

In [ ]:
#Función para crear un df que contenga solo las filas con sitios concretos
def get_30_df(df_sheet):
    starting_row=len(df_sheet)-row_count #quiero saber a partir de qué fila se han incluido las proteínas con menor ratio (log2). Ej. 126 filas en total - 11 filas parseadas (para llegar a 10 filas con fosforilación identificada) = 115 (fila en la que se empieza)
    print(len(df_sheet)-1)
    df_10=pd.DataFrame(columns=col_names) #creo un df al que voy a añadir las 10 prot. con menor log2
    df_10_row_count=0
    for i,(index,row) in enumerate(df_sheet.iterrows()):
        if i>=starting_row:
            if i in n_i_rows:
                print(str(i)+" not included")
            if i not in n_i_rows:
                print(i)
                df_10.loc[df_10_row_count]=row
                df_10_row_count+=1
    df_30=pd.concat([df_20,df_10])
    return df_30
df_30=get_30_df(df_sheet2)
df_30

In [ ]:
def get_col_names(df_sheet):
    col_names=[]
    axis_names=[]
    for name in df_sheet.columns:
        if "Abundances (Scaled)" in name:
            if "Control, IS" not in name:
                col_names.append(name)
                axis_name=name.replace('Abundances (Scaled): F1: ','').replace('Sample, ','')
                axis_names.append(axis_name) 
    return col_names, axis_names
col_names, axis_names=get_col_names(df_sheet2)
print(col_names)
print(axis_names)

In [ ]:
def get_data(df_sheet):
    df_data=df_sheet[col_names]
    #quito la "fila" de títulos y convierto la tabla en un array de valores
    data=df_data.values
    #paso lo valores a int
    data=data.astype(int)
    return data
data=get_data(df_30)
data

In [ ]:
#Función para obtener la lista de nombres de los genes
def get_gene_names(df_sheet):
    descripts_list=df_sheet['Master Protein Descriptions'].tolist()
    gene_names=[]
    descript_count=1
    #n_i_count=0
    #n_i_list=[]
    for descript in descripts_list:
        #print(descript)
        gene=descript.split("GN=")[1].split(' ')[0]
        gene_names.append(gene)
        descript_count=descript_count+1
    #n_i_perc=round(n_i_count*100/(descript_count-1),2)
    #print(gene_names)
    #print(n_i_perc)
    #print(descript_count)
    return gene_names
gene_names=get_gene_names(df_30)
print(gene_names)
print(len(gene_names))

In [ ]:
#Función para juntar genes y mods
def get_genes_w_mods(df_sheet):
    genes_w_mods=[]
    mod_counter=0
    #Uno lo lista de modificaciones de las 20 prot. con la lista de las 10 prot.
    for site in sites_list2:
        sites_list.append(site)
    for name in gene_names:
        if mod_counter<30:
            mod=sites_list[mod_counter]
            gene_w_mods=name+' '+mod
            genes_w_mods.append(gene_w_mods)
            mod_counter=mod_counter+1
    return genes_w_mods
genes_w_mods=get_genes_w_mods(df_30)
print(genes_w_mods)

In [ ]:
for i, sheet in enumerate(df_sheet_list):
    sites_list, n_i_rows, n_i_accs, perc_n_i, row_count=get_pos_mod_sites(sheet)
    print('\n'+'Accessions/genes where the exact phosphorylation site has not been identified:'+'\n'+str(n_i_accs)+' ('+str(len(n_i_accs))+')')
    df_20,col_names=get_20_df(sheet)
    sites_list2, n_i_rows, n_i_accs2, perc_n_i, row_count=get_neg_mod_sites(sheet)
    print(str(n_i_accs2)+' ('+str(len(n_i_accs2))+')'+'\n')
    df_30=get_30_df(sheet)
    col_names, axis_names=get_col_names(sheet)
    data=get_data(df_30)
    gene_names=get_gene_names(df_30)
    genes_w_mods=get_genes_w_mods(df_30)
    # Generate the clustermap
    sns.set_context("paper", font_scale=1)
    g = sns.clustermap(data, xticklabels=axis_names, yticklabels=genes_w_mods, cmap='RdBu_r', col_cluster=False) #Quitamos cbar y el dendrograma de las filas para generar unos propios
    # Increase the font size of x-axis labels
    plt.setp(g.ax_heatmap.get_xticklabels(), fontsize=10)
    # Remove the original colorbar
    g.cax.set_visible(False)
    # Create a new axes for colorbar at [left, bottom, width, height]
    # Adjust the values to place the colorbar at the top left corner and make it thicker
    cbar_ax = g.fig.add_axes([.025, .95, .2, .015])  # change these values as needed
    # Create a horizontal colorbar
    norm = plt.Normalize(np.min(data), np.max(data))
    sm = plt.cm.ScalarMappable(cmap='RdBu_r', norm=norm)
    sm.set_array([])
    g.fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
    # Set colorbar title and increase its font size
    cbar_ax.set_title('Abundance', fontsize=8, loc='center')
    cbar_ax.xaxis.set_tick_params(labelsize=6)
    # Save the figure to a PDF file
    g.savefig("heatmap_30_" + str(i) + ".pdf")